In [2]:
import pandas as pd
import tcrgp
import numpy as np

In [5]:
data = pd.read_csv('/home/alex/PycharmProjects/vdjdb-classifier-benchmark/vdjdb-dump/vdjdb_full.txt', sep='\t')

/home/alex/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (21,26,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
data.head()

,gene,cdr3.alpha,v.alpha,j.alpha,cdr3.beta,v.beta,d.beta,j.beta,species,mhc.a,...,meta.replica.id,meta.clone.id,meta.epitope.id,meta.tissue,meta.donor.MHC,meta.donor.MHC.method,meta.structure.id,cdr3fix.alpha,cdr3fix.beta,vdjdb.score
0,TRB,NaN,NaN,NaN,CSVWGTGKTYEQYF,TRBV29-1*01,NaN,TRBJ2-7*01,HomoSapiens,HLA-B*08,...,primary,NaN,NaN,NaN,"A1,26;B8,51;Cw7,15",NaN,NaN,NaN,"{""cdr3"": ""CSVWGTGKTYEQYF"", ""cdr3_old"": ""CSVWGT...",1
1,TRB,NaN,NaN,NaN,CSVWGEGRSYEQYF,TRBV29-1*01,NaN,TRBJ2-7*01,HomoSapiens,HLA-B*08,...,primary,NaN,NaN,NaN,"A1,26;B8,51;Cw7,15",NaN,NaN,NaN,"{""cdr3"": ""CSVWGEGRSYEQYF"", ""cdr3_old"": ""CSVWGE...",1
2,TRB,NaN,NaN,NaN,CSATILAGVPYGEQYF,TRBV20-1*01,NaN,TRBJ2-7*01,HomoSapiens,HLA-B*08,...,primary,NaN,NaN,NaN,"A1,26;B8,51;Cw7,15",NaN,NaN,NaN,"{""cdr3"": ""CSATILAGVPYGEQYF"", ""cdr3_old"": ""CSAT...",1
3,TRB,NaN,NaN,NaN,CSATILAGVPYGGQHF,TRBV20-1*01,NaN,TRBJ2-7*01,HomoSapiens,HLA-B*08,...,primary,NaN,NaN,NaN,"A1,26;B8,51;Cw7,15",NaN,NaN,NaN,"{""cdr3"": ""CSATILAGVPYGGQHF"", ""cdr3_old"": ""CSAT...",0
4,TRB,NaN,NaN,NaN,CSASEGTSSYEQYF,TRBV20-1*01,NaN,TRBJ2-7*01,HomoSapiens,HLA-B*08,...,primary,NaN,NaN,NaN,"A1,26;B8,51;Cw7,15",NaN,NaN,NaN,"{""cdr3"": ""CSASEGTSSYEQYF"", ""cdr3_old"": ""CSASEG...",0


In [26]:
alpha = data.loc[data['cdr3.beta'].isna()]
beta = data.loc[~data['cdr3.beta'].isna()]

In [27]:
# ['CDR3', 'V', 'J', 'Reference', 'meta.subject.id']
# v.alpha, v.beta
columns = ['species', 'gene', 'antigen.epitope', 'meta.subject.id', 'reference.id', 'v', 'j', 'cdr3']

In [28]:
chains = ['alpha', 'beta']
rename_columns = ['cdr3', 'v', 'j']
data_genes = [alpha, beta]
for i, chain in enumerate(chains):
    chain = chains[i]
    old = [f'{x}.{chain}' for x in rename_columns]
    new = [f'{x}' for x in rename_columns]
    mapper = {x: y for (x, y) in zip(old, new)}
    data_genes[i].rename(columns=mapper, inplace=True)
    drop = data_genes[i].columns.difference(columns)
    data_genes[i].drop(drop, axis=1, inplace=True)

In [24]:
alpha.columns.difference(columns)

Index(['antigen.gene', 'antigen.species', 'cdr3.beta', 'cdr3fix.alpha',
       'cdr3fix.beta', 'd.beta', 'j.beta', 'meta.cell.subset', 'meta.clone.id',
       'meta.donor.MHC', 'meta.donor.MHC.method', 'meta.epitope.id',
       'meta.replica.id', 'meta.structure.id', 'meta.study.id',
       'meta.subject.cohort', 'meta.tissue', 'method.frequency',
       'method.identification', 'method.sequencing', 'method.singlecell',
       'method.verification', 'mhc.a', 'mhc.b', 'mhc.class', 'v.beta',
       'vdjdb.score'],
      dtype='object')

In [33]:
data = pd.concat([alpha, beta])

In [34]:
data.head()

,gene,cdr3,v,j,species,antigen.epitope,reference.id,meta.subject.id
162,TRA,CAVGNNAGNMLTF,TRAV20*01,TRAJ39*01,HomoSapiens,NEGVKAAW,PMID:30487790,30
163,TRA,CAAKTGGGNKLTF,TRAV29/DV5*01,TRAJ10*01,HomoSapiens,NEGVKAAW,PMID:30487790,30
164,TRA,CAAKAGGGNKLTF,TRAV29/DV5*01,TRAJ10*01,HomoSapiens,NEGVKAAW,PMID:30487790,30
165,TRA,CAAITGIGGSQGNLIF,TRAV13-1*01,TRAJ42*01,HomoSapiens,NEGVKAAW,PMID:30487790,30
166,TRA,CAGTDGGATNKLIF,TRAV36/DV7*01,TRAJ32*01,HomoSapiens,NEGVKAAW,PMID:30487790,30


In [37]:
data['reference.id'] = data['reference.id'] + '_' + data['meta.subject.id']

In [38]:
data.drop('meta.subject.id', axis=1, inplace=True)

In [41]:
mapper = {'species': 'Species', 'gene': 'Gene', 'antigen.epitope': 'Epitope', 
          'cdr3': 'CDR3', 'v': 'V', 'j':'J', 'reference.id': 'Reference'}
data.rename(columns=mapper, inplace=True)

In [42]:
data.to_csv('tcrgp_vdjdb.csv', index=False)